# Biome Level statistics for model: GS2_MeanScaler


In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [2]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()


In [3]:
# initialize the earth engine API
ee.Initialize()

## 1 Load the required composites and images

In [4]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the root shoot ratio map
rootShootRatio = ee.Image("users/leonidmoore/ForestBiomass/Root_shoot_ratio_Map").unmask()
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## 2 Load the biomass density maps

In [5]:
# load the carbon density layers
potentialDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS2_MeanScaler_Potential_density_Ensambled_Mean_20230427").unmask()
presentDensity = ee.Image("users/leonidmoore/ForestBiomass/GroundSourcedModel/EnsambledMaps/Predicted_GS1_MeanScaler_Present_density_Ensambled_Mean_20230427").unmask()

# define the standard projection
stdProj = potentialDensity.projection();

## 3 Adjust the present and potential density maps

In [6]:
# load the present and potential forest cover
presentForestCover = compositeImage.select('PresentTreeCover').unmask() # uniform with potential in the  0-1 scale
potentialCoverAdjusted = ee.Image("users/leonidmoore/ForestBiomass/Bastin_et_al_2019_Potential_Forest_Cover_Adjusted").unmask().rename('PotentialForestCover')
# define the present and potential forest cover masks
presentMask= presentForestCover.gt(0)
potentialMask= potentialCoverAdjusted.gt(0)

# check the difference of the two density maps
potentialHigher = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).gte(0)
potentialLower = potentialDensity.multiply(pixelAreaMap).subtract(presentDensity.multiply(pixelAreaMap)).lt(0)
# replace the lower potential value by present biomass density value
agbPotentialDensity = presentDensity.multiply(potentialLower).add(potentialDensity.multiply(potentialHigher))
# add the root biomass to the AGB to get TGB
tgbPotentialDensity = agbPotentialDensity.multiply(rootShootRatio).add(agbPotentialDensity)
tgbPresentDensity = presentDensity.multiply(rootShootRatio).add(presentDensity)

## 4 Partioning the potential cover into different landuse types

In [7]:
# Load all the landuse type layers
croplandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/cropland_Percent").rename('cropland').divide(100).reproject(crs=stdProj);
grazingOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/grazing_Percent").rename('grazing').divide(100).reproject(crs=stdProj);
pastureOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/pasture_Percent").rename('pasture').divide(100).reproject(crs=stdProj);
rangelandOrg = ee.Image("users/leonidmoore/ForestBiomass/HYDE31/rangeland_Percent").rename('rangeland').divide(100).reproject(crs=stdProj);
urbanOrg = compositeImage.select(['LandCoverClass_Urban_Builtup']).divide(100).unmask().reproject(crs=stdProj);
snowIceOrg = compositeImageNew.select(['ConsensusLandCoverClass_Snow_Ice']).divide(100).unmask().reproject(crs=stdProj);
openWaterOrg = compositeImageNew.select(['ConsensusLandCoverClass_Open_Water']).divide(100).unmask().reproject(crs=stdProj);
# define the total landcover types
sumCover = presentForestCover.add(pastureOrg).add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(openWaterOrg).add(snowIceOrg);
oneSubtract = ee.Image(1).subtract(sumCover);
freeland = oneSubtract.multiply(oneSubtract.gte(0));
# get the scale ratio for pixels with sumCover larger than 1
scaleRatio = ee.Image(1).subtract(presentForestCover).divide(sumCover.subtract(presentForestCover)).multiply(oneSubtract.lt(0));
# get the ratio of these three disturbed maps
pasture = pastureOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(pastureOrg.multiply(oneSubtract.gte(0))).unmask();
rangeland = rangelandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(rangelandOrg.multiply(oneSubtract.gte(0))).unmask();
cropland = croplandOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(croplandOrg.multiply(oneSubtract.gte(0))).unmask();
urban = urbanOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(urbanOrg.multiply(oneSubtract.gte(0))).unmask();
openWater = openWaterOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(openWaterOrg.multiply(oneSubtract.gte(0))).unmask();
snowIce = snowIceOrg.multiply(scaleRatio).multiply(oneSubtract.lt(0)).add(snowIceOrg.multiply(oneSubtract.gte(0))).unmask();
sumTT = presentForestCover.add(pasture).add(rangeland).add(cropland).add(urban).add(freeland).add(openWater).add(snowIce).unmask();

effectivePotentialMask = freeland.add(rangeland).add(pasture).add(cropland).add(urban).gt(0);
# there are some pixels without any landcover survived but with open water and ice and snow. here we mask these pixels out
sumlandCover = pastureOrg.add(rangelandOrg).add(croplandOrg).add(urbanOrg).add(freeland);
restorationMap = potentialCoverAdjusted.subtract(presentForestCover).mask(effectivePotentialMask).unmask();

# sum all these scaled layersv
scaledSum = pasture.add(rangeland).add(cropland).add(urban).add(freeland);
potentialCoverFinal = restorationMap.add(presentForestCover);
# allocate the potential equally to each layer
freelandPotentialCover = freeland.divide(scaledSum).multiply(restorationMap).unmask();
rangelandPotentialCover = rangeland.divide(scaledSum).multiply(restorationMap).unmask();
pasturePotentialCover = pasture.divide(scaledSum).multiply(restorationMap).unmask();
croplandPotentialCover = cropland.divide(scaledSum).multiply(restorationMap).unmask();
urbanPotentialCover = urban.divide(scaledSum).multiply(restorationMap).unmask();
#  allocate the freeland potential in pixels with forest cover larger than 10% to conservation potential
freelandForConsevation = freelandPotentialCover.multiply(presentForestCover.gte(0.1)).unmask();
maximumPotentialCover = freelandForConsevation.add(presentForestCover);
# calucate the reall freeland outside of forest
freelandLeftMap = freelandPotentialCover.subtract(freelandForConsevation).unmask()# the left positive pixels are real freeland pixels

## 5 Partioning the biomass potential into different landuse types

In [8]:
# calculate the existing carbon, present potential carbon and absolute potential carbon in forests
absoluteImage1 = tgbPresentDensity.multiply(pixelAreaMap).multiply(presentMask).divide(1000000000).rename('Present')
absoluteImage3 = tgbPotentialDensity.multiply(pixelAreaMap).multiply(potentialCoverFinal.gt(0)).divide(1000000000).rename('AbsolutePotential')
# get the sum of the potential covers
potentialCoverSum = freelandLeftMap.add(rangelandPotentialCover).add(pasturePotentialCover).add(croplandPotentialCover).add(urbanPotentialCover)

trueRestorationPotential = absoluteImage3.subtract(absoluteImage1).multiply(1000000000)
ratioPotentialBiomassDensity = absoluteImage1.multiply(potentialCoverFinal.divide(presentForestCover))
#  get the real for conservation potential
realDensityIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).gt(0)).unmask()
realDensityNotIncreased = absoluteImage3.subtract(absoluteImage1).mask(absoluteImage3.subtract(ratioPotentialBiomassDensity).lte(0)).unmask()
trueReforestationPotential = realDensityNotIncreased.add(realDensityIncreased.multiply(ee.Image(1).subtract(presentForestCover.divide(potentialCoverFinal))))

conservationPotentialPart1 = realDensityIncreased.multiply(presentForestCover.add(freelandForConsevation).divide(potentialCoverFinal))
conservationPotentialPart2 = realDensityNotIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal.subtract(presentForestCover)))

# calculate the part of the potential inside the forest cover which was allocate to conservation potential.
freelandForConservation1 = realDensityIncreased.multiply(freelandForConsevation.divide(potentialCoverFinal))
freelandForConservation = freelandForConservation1.add(conservationPotentialPart2).rename('FreeToConservation')

absoluteImage2 = conservationPotentialPart1.add(conservationPotentialPart2).add(absoluteImage1).rename('PresentPotential')

trueReforestationPotential = absoluteImage3.subtract(absoluteImage2)

absoluteImage4 = trueReforestationPotential.multiply(freelandLeftMap.divide(potentialCoverSum)).rename('FreelandPotential')
absoluteImage5 = trueReforestationPotential.multiply(rangelandPotentialCover.divide(potentialCoverSum)).rename('RangelandPotential')
absoluteImage6 = trueReforestationPotential.multiply(pasturePotentialCover.divide(potentialCoverSum)).rename('PasturePotential')
absoluteImage7 = trueReforestationPotential.multiply(croplandPotentialCover.divide(potentialCoverSum)).rename('CroplandPotential')
absoluteImage8 = trueReforestationPotential.multiply(urbanPotentialCover.divide(potentialCoverSum)).rename('UrbanPotential')

## Calculate the potential numbers and write into local folder

In [11]:
# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum() 
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/GS2_MeanScaler_Biome_Level_Statistics.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,233.543439,269.806630,299.524066,5.798349,0.091919,10.607761,12.855956,0.363451,8.596743
1,7.873063,12.294854,21.424844,2.278476,0.019752,2.352541,4.426988,0.052233,1.073360
2,3.074780,5.117734,6.364809,0.450744,0.019277,0.438652,0.333811,0.004591,0.540396
3,40.546435,51.932885,71.073962,4.440548,0.024344,5.681818,8.449535,0.544833,3.997965
4,19.024499,24.494401,27.494649,1.370050,0.035043,0.894245,0.642814,0.058096,2.123871
5,29.181260,31.599380,33.629771,1.755061,0.000780,0.136060,0.129240,0.009250,1.709853
6,47.409114,83.948620,122.215941,6.901612,12.284768,11.286027,7.707090,0.087824,6.692740
7,3.654017,4.803653,18.965463,2.489417,4.180546,1.944288,5.461200,0.086358,0.232306
8,1.732141,3.047002,4.337056,0.335463,0.504394,0.275864,0.163549,0.010784,0.198462
9,3.527103,5.620812,9.221342,0.849498,1.195238,0.758131,0.789972,0.007690,0.356904


In [28]:
deadWoodLitterRatio = ee.Image("users/leonidmoore/ForestBiomass/DeadWoodLitter/DeadWood_Litter_Ratio_Map").unmask()

# Stack the absolute biomass layers into an Image.
absPotentialImage = absoluteImage1.addBands(absoluteImage2).addBands(absoluteImage3).addBands(absoluteImage4).addBands(absoluteImage5).addBands(absoluteImage6).addBands(absoluteImage7).addBands(absoluteImage8).addBands(freelandForConservation)
# define the function to do the biome level statistics which could be applied by map      
def biomeLevelStat(biome):
    biomeMask = biomeLayer.eq(ee.Number(biome))
    masked_img = absPotentialImage.mask(biomeMask).multiply(deadWoodLitterRatio)
    output = masked_img.reduceRegion(reducer= ee.Reducer.sum(),
                                     geometry= unboundedGeo,
                                     crs='EPSG:4326',
                                     crsTransform=[0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                     maxPixels= 1e13)
    return output#.getInfo().get('Present')


biomeList = ee.List([1,2,3,4,5,6,7,8,9,10,11,12,13,14])
statisticTable = biomeList.map(biomeLevelStat).getInfo()
# transform into data frame
outputTable = pd.DataFrame(statisticTable,columns =['Present','PresentPotential','AbsolutePotential','FreelandPotential','RangelandPotential','PasturePotential','CroplandPotential','UrbanPotential','FreeToConservation'])#.round(1)
outputTable.loc['sum'] = outputTable.sum()
outputTable.to_csv('Data/BiomeLevelStatistics/StatisticsForModels/GS2_MeanScaler_Biome_Level_Statistics_with_Litter.csv',header=True,mode='w+')
# display the output of the carbon partitioning
print(colored('The biomass partition results in biome: \n', 'blue', attrs=['bold']))
outputTable.head(15)

The biomass partition results in biome: 



,Present,PresentPotential,AbsolutePotential,FreelandPotential,RangelandPotential,PasturePotential,CroplandPotential,UrbanPotential,FreeToConservation
0,284.933366,329.178901,365.437302,7.074564,0.112343,12.942542,15.685541,0.443412,10.488907
1,9.605110,14.999663,26.138329,2.779741,0.024190,2.870126,5.400884,0.063724,1.309489
2,3.751965,6.245064,7.766723,0.550017,0.023492,0.535142,0.407404,0.005603,0.659551
3,53.922454,69.063279,94.517776,5.904689,0.032331,7.556168,11.236764,0.724544,5.316117
4,25.301132,32.574769,36.562829,1.821039,0.046350,1.188979,0.854465,0.077227,2.824089
5,52.498388,56.848310,60.499426,3.157440,0.001312,0.244352,0.231403,0.016609,3.075938
6,57.839603,102.418085,149.104943,8.420035,14.987762,13.769222,9.402690,0.107149,8.165158
7,4.860284,6.389275,25.223478,3.310744,5.559504,2.585767,7.263339,0.114850,0.308994
8,2.113578,3.717812,5.291914,0.409329,0.615372,0.336609,0.199622,0.013169,0.242162
9,4.680865,7.462544,12.247763,1.128875,1.588721,1.007492,1.049910,0.010222,0.474094


In [11]:
# If you got the error 'EEException: Too many concurrent aggregations.', please re-run this chunck of code again.